## Geöffnete Skilifte absaugen

In [1]:
import requests
website = requests.get("https://www.bergfex.ch/ybrig/").text

In [2]:
website

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n\n<html>\n<head>\n  <title>BERGFEX: Skigebiet Hoch-Ybrig - Skiurlaub Hoch-Ybrig - Wintersportgebiet Hoch-Ybrig</title>\n  <meta name="keywords" content="Skifahren,Skigebiet,Skiurlaub,nolang:Einsiedeln,nolang:Unteriberg,nolang:Oberiberg,nolang:Hoch-Ybrig,nolang:">\n  <meta name="description" content="Skigebiet Hoch-Ybrig - Luzern - Vierwaldstättersee - Schweiz: Alle Informationen zum Skiurlaub Hoch-Ybrig. Sie erhalten Wintersport-Infos zum Skifahren, Pistenplan, Bildern, Preisen,  Schneeh�hen, Hotels, Kontakt, Veranstaltungen u.v.m.">\n\n  <meta name="ICBM" content="47.011221, 8.784342"/>\n  <meta name="geo.position" content="47.011221;8.784342"/>\n  <meta name="geo.region" content="CH"/>\n  <meta name="geo.placename" content="Skigebiet Hoch-Ybrig"/>\n\n  <link rel="location" name="Schweiz" href="/schweiz/">\n  <link rel="location" name="Luzern - Vierwaldstättersee" href="/luz

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website, "html.parser")

In [8]:
offene_skilifte = []
for skigebiet in soup.find_all('table', class_="schneewerte"):
    print(skigebiet.td)
    #srftitel.append({"Titel":titel.text.strip()})
    prePostBody.find("table", {"class": "tborder"}).find("tr", {"valign": "top"}).find("td", {"class": "alt1"})
    print(prePostBody)

<td style="font-size: 12px; padding-top: 3px;">Heute, 08:19</td>
